In [9]:
#Import the necessary libraries

import cv2
import numpy as np
from ultralytics import YOLO
from pathlib import Path
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import yaml
import shutil
import os

In [10]:
def prepare_yolo_dataset(dataset_path, output_path="xray_yolo_dataset", test_split=0.2):
    print("="*60)
    print("📦 PREPARING YOLO DATASET")
    print("="*60)
    
    dataset_path = Path(dataset_path)
    output_path = Path(output_path)
    
    # Verify dataset path exists
    if not dataset_path.exists():
        raise FileNotFoundError(f"❌ Dataset path does not exist: {dataset_path}")
    
    print(f"\n📁 Dataset path: {dataset_path.absolute()}")
    
    # Create YOLO directory structure
    (output_path / "images" / "train").mkdir(parents=True, exist_ok=True)
    (output_path / "images" / "val").mkdir(parents=True, exist_ok=True)
    (output_path / "labels" / "train").mkdir(parents=True, exist_ok=True)
    (output_path / "labels" / "val").mkdir(parents=True, exist_ok=True)
    
    # Get class names from folder structure
    class_names = sorted([d.name for d in dataset_path.iterdir() if d.is_dir()])
    
    if not class_names:
        raise ValueError(f"❌ No class folders found in {dataset_path}!\n"
                        "Make sure your dataset has subfolders for each class.")
    
    print(f"\n📋 Classes found: {class_names}")
    
    class_to_idx = {name: idx for idx, name in enumerate(class_names)}
    
    # Process each class
    all_images = []
    
    for class_name in class_names:
        class_folder = dataset_path / class_name
        class_idx = class_to_idx[class_name]
        
        # Get all images (handle both uppercase and lowercase extensions)
        image_files = (
            list(class_folder.glob("*.jpg")) +
            list(class_folder.glob("*.JPG")) +
            list(class_folder.glob("*.png")) +
            list(class_folder.glob("*.PNG")) +
            list(class_folder.glob("*.jpeg")) +
            list(class_folder.glob("*.JPEG"))
        )
        
        print(f"   {class_name}: {len(image_files)} images")
        
        if len(image_files) == 0:
            print(f"      ⚠️  WARNING: No images found in {class_name}/ folder!")
        
        for img_file in image_files:
            all_images.append((img_file, class_idx, class_name))
    
    # Check if we found any images
    if len(all_images) == 0:
        raise ValueError(
            f"❌ No images found in dataset!\n\n"
            f"Checked path: {dataset_path.absolute()}\n"
            f"Found folders: {class_names}\n\n"
            f"Make sure:\n"
            f"  1. Images are inside the class folders\n"
            f"  2. Images have extensions: .jpg, .JPG, .png, .PNG, .jpeg, .JPEG\n"
            f"  3. Path is correct"
        )
    
    print(f"\n✅ Total images found: {len(all_images)}")
    
    # Split into train and validation
    train_data, val_data = train_test_split(
        all_images, 
        test_size=test_split, 
        random_state=42,
        stratify=[x[1] for x in all_images]
    )
    
    print(f"\n✅ Split complete:")
    print(f"   Training: {len(train_data)} images")
    print(f"   Validation: {len(val_data)} images")
    
    # Copy files and create labels
    print("\n📁 Copying files and creating labels...")
    
    def process_split(data, split_name):
        for img_path, class_idx, class_name in data:
            # Copy image
            new_img_name = f"{class_name}_{img_path.stem}.jpg"
            new_img_path = output_path / "images" / split_name / new_img_name
            shutil.copy(img_path, new_img_path)
            
            # Create label file (YOLO format: class x_center y_center width height)
            # For classification, we assume the entire image is the object
            label_path = output_path / "labels" / split_name / f"{new_img_name.replace('.jpg', '.txt')}"
            
            # YOLO format: normalized coordinates
            # For full image classification: center at 0.5, 0.5, full width/height = 1.0
            with open(label_path, 'w') as f:
                f.write(f"{class_idx} 0.5 0.5 1.0 1.0\n")
    
    process_split(train_data, "train")
    process_split(val_data, "val")
    
    print("✅ Files copied and labels created!")
    
    # Create data.yaml file
    data_yaml = {
        'path': str(output_path.absolute()),
        'train': 'images/train',
        'val': 'images/val',
        'nc': len(class_names),  # number of classes
        'names': class_names
    }
    
    yaml_path = output_path / "data.yaml"
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f)
    
    print(f"\n✅ data.yaml created at: {yaml_path}")
    
    return class_names, str(yaml_path)

In [11]:

# ============================================
# STEP 2: MODEL TRAINING
# ============================================

def train_yolo_model(data_yaml_path, epochs=50, img_size=640, batch_size=16, model_name="yolov8n.pt"):    
    print("\n" + "="*60)
    print("🚀 STARTING YOLO TRAINING")
    print("="*60)
    
    print(f"\n📊 Training Configuration:")
    print(f"   Model: {model_name}")
    print(f"   Epochs: {epochs}")
    print(f"   Image Size: {img_size}")
    print(f"   Batch Size: {batch_size}")
    
    # Load pre-trained model
    print(f"\n📦 Loading pre-trained {model_name}...")
    model = YOLO(model_name)
    
    # Train the model
    print("\n🏋️ Training started...\n")
    
    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        imgsz=img_size,
        batch=batch_size,
        patience=10,  # Early stopping patience
        save=True,
        device='cpu',  # Change to 'cuda' or 'mps' if you have GPU
        workers=2,
        project='xray_runs',
        name='xray_detector',
        exist_ok=True,
        verbose=True
    )
    
    print("\n" + "="*60)
    print("✅ TRAINING COMPLETE!")
    print("="*60)
    
    return model, results

In [12]:
# ============================================
# STEP 3: MODEL EVALUATION
# ============================================

def evaluate_model(model, data_yaml_path, img_size=640):
    print("\n" + "="*60)
    print("📊 EVALUATING MODEL")
    print("="*60)
    
    # Validate the model
    metrics = model.val(
        data=data_yaml_path,
        imgsz=img_size,
        batch=16,
        device='cpu',
        verbose=True
    )
    
    # Print metrics
    print("\n📈 Validation Results:")
    print(f"   Precision: {metrics.box.mp:.4f}")
    print(f"   Recall: {metrics.box.mr:.4f}")
    print(f"   mAP@50: {metrics.box.map50:.4f}")
    print(f"   mAP@50-95: {metrics.box.map:.4f}")
    
    return metrics

In [13]:
# ============================================
# STEP 4: TEST ON SAMPLE IMAGES
# ============================================

def test_on_images(model, test_images_path, class_names, confidence_threshold=0.5):

    print("\n" + "="*60)
    print("🧪 TESTING ON SAMPLE IMAGES")
    print("="*60)
    
    test_path = Path(test_images_path)
    image_files = list(test_path.glob("*.jpg")) + \
                 list(test_path.glob("*.png")) + \
                 list(test_path.glob("*.jpeg"))
    
    if not image_files:
        print("❌ No test images found!")
        return
    
    print(f"\n📸 Found {len(image_files)} test images")
    
    # Test on each image
    for i, img_path in enumerate(image_files[:10], 1):  # Test first 10 images
        print(f"\n{i}. Testing: {img_path.name}")
        
        # Run inference
        results = model(str(img_path), conf=confidence_threshold, verbose=False)
        
        # Get predictions
        for result in results:
            boxes = result.boxes
            
            if len(boxes) > 0:
                # Get the most confident detection
                conf = boxes.conf[0].item()
                cls = int(boxes.cls[0].item())
                class_name = class_names[cls]
                
                print(f"   ✅ Detected: {class_name} ({conf*100:.1f}% confidence)")
                
                # Display image with detection
                img = result.plot()
                cv2.imshow(f'Test Result - {img_path.name}', img)
                cv2.waitKey(1000)  # Show for 1 second
            else:
                print(f"   ❌ No detections above {confidence_threshold} confidence")
    
    cv2.destroyAllWindows()


def visualize_predictions(model, test_images_path, class_names, num_samples=6):
    
    print("\n📊 Creating prediction visualization...")
    
    test_path = Path(test_images_path)
    image_files = list(test_path.glob("*.jpg"))[:num_samples]
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    for idx, img_path in enumerate(image_files):
        # Run inference
        results = model(str(img_path), verbose=False)
        
        # Get annotated image
        img = results[0].plot()
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Display
        axes[idx].imshow(img_rgb)
        axes[idx].axis('off')
        axes[idx].set_title(f'{img_path.name}', fontsize=10)
    
    plt.tight_layout()
    plt.savefig('prediction_samples.png', dpi=150, bbox_inches='tight')
    print("✅ Visualization saved as 'prediction_samples.png'")
    plt.show()

In [14]:


# ============================================
# STEP 5: SAVE MODEL
# ============================================

def save_model(model, save_path="xray_threat_detector.pt"):
    
    print(f"\n💾 Saving model to: {save_path}")
    
    # Export model
    model.export(format='torchscript', save_dir=Path(save_path).parent)
    
    # Also save as .pt
    shutil.copy(
        model.trainer.best,  # Best weights from training
        save_path
    )
    
    print(f"✅ Model saved!")
    print(f"   Location: {Path(save_path).absolute()}")

In [15]:
# ============================================
# COMPLETE PIPELINE
# ============================================

def run_complete_pipeline(
    dataset_path,
    epochs=50,
    img_size=640,
    batch_size=16,
    model_variant="yolov8m.pt"
):
    
    print("""
    ╔══════════════════════════════════════════════════════════╗
    ║                                                          ║
    ║         X-RAY THREAT DETECTION - PHASE 1                 ║
    ║              YOLOv8 Training Pipeline                    ║
    ║                                                          ║
    ╚══════════════════════════════════════════════════════════╝
    """)
    
    start_time = time.time()
    
    # Step 1: Prepare dataset
    class_names, data_yaml_path = prepare_yolo_dataset(dataset_path)
    
    # Step 2: Train model
    model, training_results = train_yolo_model(
        data_yaml_path,
        epochs=epochs,
        img_size=img_size,
        batch_size=batch_size,
        model_name=model_variant
    )
    
    # Step 3: Evaluate model
    metrics = evaluate_model(model, data_yaml_path, img_size=img_size)
    
    # Step 4: Test on sample images
    val_images_path = Path("xray_yolo_dataset/images/val")
    test_on_images(model, val_images_path, class_names)
    visualize_predictions(model, val_images_path, class_names)
    
    # Step 5: Save model
    save_model(model, "xray_threat_detector_best.pt")
    
    # Summary
    elapsed_time = time.time() - start_time
    
    print("\n" + "="*60)
    print("🎉 PIPELINE COMPLETE!")
    print("="*60)
    print(f"\n⏱️  Total Time: {elapsed_time/60:.1f} minutes")
    print(f"\n📊 Final Metrics:")
    print(f"   Classes: {class_names}")
    print(f"   Training Images: Shown above")
    print(f"   Validation mAP@50: {metrics.box.map50:.4f}")
    print(f"   Model saved: xray_threat_detector_best.pt")
    
    print("\n📁 Output Files:")
    print("   - xray_yolo_dataset/          (Formatted dataset)")
    print("   - xray_runs/                  (Training logs)")
    print("   - xray_threat_detector_best.pt (Trained model)")
    print("   - prediction_samples.png      (Sample predictions)")
    
    print("\n🚀 Ready for Phase 2: Video Processing!")

from pathlib import Path

# Update this to your actual path
DATASET_PATH = "Baggages"

dataset_path = Path(DATASET_PATH)

print("="*60)
print("🔍 DATASET DIAGNOSTIC")
print("="*60)

print(f"\n📁 Checking path: {dataset_path.absolute()}")
print(f"   Path exists: {dataset_path.exists()}")
print(f"   Is directory: {dataset_path.is_dir()}")

if dataset_path.exists():
    print("\n📂 Contents of dataset folder:")
    
    for item in dataset_path.iterdir():
        if item.is_dir():
            # Count images in this folder
            jpg_count = len(list(item.glob("*.jpg")))
            png_count = len(list(item.glob("*.png")))
            jpeg_count = len(list(item.glob("*.jpeg")))
            JPG_count = len(list(item.glob("*.JPG")))  # Uppercase
            
            total = jpg_count + png_count + jpeg_count + JPG_count
            
            print(f"   📁 {item.name}/")
            print(f"      .jpg:  {jpg_count}")
            print(f"      .JPG:  {JPG_count}")
            print(f"      .png:  {png_count}")
            print(f"      .jpeg: {jpeg_count}")
            print(f"      TOTAL: {total} images")
        else:
            print(f"   📄 {item.name} (file, not folder)")
else:
    print("\n❌ Path does NOT exist!")
    print("💡 Make sure you updated DATASET_PATH correctly")

print("\n" + "="*60)

🔍 DATASET DIAGNOSTIC

📁 Checking path: c:\Users\Enzo\Desktop\CODING PROJECTS\Python Exercises\Mini_Rapiscan920CT\Baggages
   Path exists: True
   Is directory: True

📂 Contents of dataset folder:
   📁 Blade/
      .jpg:  0
      .JPG:  0
      .png:  100
      .jpeg: 0
      TOTAL: 100 images
   📁 Gun/
      .jpg:  0
      .JPG:  0
      .png:  100
      .jpeg: 0
      TOTAL: 100 images
   📁 Knife/
      .jpg:  0
      .JPG:  0
      .png:  100
      .jpeg: 0
      TOTAL: 100 images
   📁 Pin/
      .jpg:  0
      .JPG:  0
      .png:  100
      .jpeg: 0
      TOTAL: 100 images
   📁 Projectile/
      .jpg:  0
      .JPG:  0
      .png:  100
      .jpeg: 0
      TOTAL: 100 images
   📁 Screw/
      .jpg:  0
      .JPG:  0
      .png:  100
      .jpeg: 0
      TOTAL: 100 images
   📁 Spring/
      .jpg:  0
      .JPG:  0
      .png:  100
      .jpeg: 0
      TOTAL: 100 images



In [16]:
if __name__ == "__main__":
    
    # Update this path to your dataset location
    DATASET_PATH = "Baggages"
    
    # Run everything
    run_complete_pipeline(
        dataset_path= DATASET_PATH,  # Small dataset!
        epochs=10,
        img_size=416,
        batch_size=8,
    model_variant="yolov8n.pt"
    )

# Should complete in 2-3 hours. THE ENTIRE PROCESS, NOT JUST AN EPOCH


    ╔══════════════════════════════════════════════════════════╗
    ║                                                          ║
    ║         X-RAY THREAT DETECTION - PHASE 1                 ║
    ║              YOLOv8 Training Pipeline                    ║
    ║                                                          ║
    ╚══════════════════════════════════════════════════════════╝
    
📦 PREPARING YOLO DATASET

📁 Dataset path: c:\Users\Enzo\Desktop\CODING PROJECTS\Python Exercises\Mini_Rapiscan920CT\Baggages

📋 Classes found: ['Blade', 'Gun', 'Knife', 'Pin', 'Projectile', 'Screw', 'Spring']
   Blade: 200 images
   Gun: 200 images
   Knife: 200 images
   Pin: 200 images
   Projectile: 200 images
   Screw: 200 images
   Spring: 200 images

✅ Total images found: 1400

✅ Split complete:
   Training: 1120 images
   Validation: 280 images

📁 Copying files and creating labels...
✅ Files copied and labels created!

✅ data.yaml created at: xray_yolo_dataset\data.yaml

🚀 STARTING YOLO TRA

<Figure size 1500x1000 with 6 Axes>


💾 Saving model to: xray_threat_detector_best.pt
Ultralytics 8.3.227  Python-3.10.5 torch-2.4.1+cu118 CPU (Intel Core i5-8265U 1.60GHz)
 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from 'C:\Users\Enzo\Desktop\CODING PROJECTS\Python Exercises\Mini_Rapiscan920CT\xray_runs\xray_detector\weights\best.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 11, 3549) (5.9 MB)

TorchScript: starting export with torch 2.4.1+cu118...
TorchScript: export success  4.6s, saved as 'C:\Users\Enzo\Desktop\CODING PROJECTS\Python Exercises\Mini_Rapiscan920CT\xray_runs\xray_detector\weights\best.torchscript' (11.8 MB)

Export complete (4.9s)
Results saved to C:\Users\Enzo\Desktop\CODING PROJECTS\Python Exercises\Mini_Rapiscan920CT\xray_runs\xray_detector\weights
Predict:         yolo predict task=detect model=C:\Users\Enzo\Desktop\CODING PROJECTS\Python Exercises\Mini_Rapiscan920